### 1. Query TD Ameritrade API

In [14]:
# Import the TD Ameritrade API client package
# !pip install td-ameritrade-python-api
from td.client import TDClient

# Create a new session; credentials path is required.
TDSession = TDClient(
    client_id = 'J4QOQPWNPV2SN0ZUYGZ4SAWFZKFMPYNX', # my developer API key
    redirect_uri = 'https://localhost',             # set in API developer account
    credentials_path = 'td_state.json'              # save in cwd for now
)

# Login to the session
TDSession.login()

True

In [15]:
# create timestamp
import datetime as dt

print(f'Data acquired at {dt.datetime.now()}.')

# Grab real-time stock quotes for 'NVDA' (Nvidia Corporation)
# nvda_quotes = TDSession.get_quotes(instruments=['NVDA'])
# pprint(nvda_quotes)

# Grab real-time option quotes
nvda_quotes_options = TDSession.get_options_chain(option_chain={'symbol': 'NVDA'})

# insert timestamp field
# nvda_quotes['timestamp'] = now
nvda_quotes_options['timestamp'] = 1400

Data acquired at 2021-12-16 13:59:15.640855.


### 2. Store JSON response in DB

In [16]:
from pymongo import MongoClient
# created 'proj7db' in mongo shell somehow
# https://www.mongodb.com/basics/create-database

# Instantiate client; confirm 'proj7db' among available databases
client = MongoClient()
client.list_database_names()

# Define working db
db = client.proj7db

# Define working collection
prices = db.prices

# Store the API response
# prices.insert_one(nvda_quotes)
prices.insert_one(nvda_quotes_options)

# Wipe the collection (if needed)
# prices.delete_many({})           # '{}' nuclear option

# Confirm one record has been added
prices.count_documents({})

6

Pipeline continued in <code>STREER.07Engineering.DataAcqusition.ipynb</code>

In [12]:
# Inspect the records here if you like though

cursor = list(prices.find({'symbol':'NVDA'}, {'_id': 0, 
                                     'symbol': 1, 
                                     'underlyingPrice': 1, 
                                     'timestamp': 1, 
                                     'callExpDateMap': 1,
                                     'putExpDateMap': 1}))

# For example, 1st record of puts for the 2021-12-17 chain:
# cursor[0]['putExpDateMap']['2021-12-17:6']

cursor[-1]['timestamp']

1300